In [1]:
import sympy as sp
import script as rt

from IPython.display import display
from IPython.lib.display import IFrame

## Општо за инверзна кинематика

Разгледуваме роботска рака со $N$ зглобови. 

Ако секој зглоб го поставиме на некоја вредност, извршниот елемент на роботската рака ќе постигне соодветна позиција и ориентација. Ова се нарекува `директна кинематика`. Со други зборови, директната кинематика одговара на прашањето `Која позиција и ориентација ќе ја заземе извршниот елемент на роботската рака ако секој зглоб го поставиме на некоја однапред определена вредност?`.

Инверзната кинематика одговара на обратното прашање. `Како да ги поставиме зглобовите на роботската рака за извршниот елемент да ја постигне посакуваната/бараната позиција или ориентација?`. Посакуваната/бараната позиција или ориентација со еден збор можеме да ја нарекуваме `барање`. Ова барање може да биде:
* Дадена позиција која треба да се постигне.
* Дадена ориентација која треба да се постигне.
* Дадена позиција и ориентација кои треба да се постигнат. Со други зборови, позната е трансформационата матрица на извршниот елемент во однос на референтниот координатен систем.
* Парцијално барање. Со други зборови, трансформационата матрица на извршниот елемент во однос на референтниот координатен систем не е целосно позната.

Следно прашање е дали роботската рака може да одоговори на барањето. Постојат неколку видови одговори на поставеното барање.
* Има едно и само едно решение на поставеното барање. Роботската рака може на само еден начин да го постави извршниот елемент според барањето.
* Има повеќе решенија, но во конечен број. Роботската рака може на неколку различни начини да го постави извршниот елемент според барањето.
* Има бесконечно многу решенија. Роботската рака може на бесконечно многу различни начини да го постави извршниот елемент според барањето.
* Нема решение. Роботската рака не може да го постави извршниот елемент според барањето.

Постојат повеќе начини како да се пресметаат вредностите на кои треба да се постават зглобовите. За ова повеќе можете да научите од книгата и предавањата. Во збирката решаваме со геометриски приод. Матрицата на директната кинематика ја добиваме од DH моделот на роботската рака. Понатаму, според поставеното барање, ги користиме изразите од DH-матрицата кои соодвествуваат на зададените вредности од барањето. Со трансформација на соодветните изрази од DH матрицата, ги изразуваме вредностите кои треба да ги заземат зглобовите.

### Задача 2.60

In [45]:
IFrame(src='https://drive.google.com/file/d/1JxA56APN-z2C6x22gPfAiaf3Vc-2meHk/preview', width='400', height='300')

Во оваа задача поставено е барање извршниот елемент да биде поставен според прикажаната трансформациона матрица. Прво ќе го пресметаме DH моделот на оваа роботска рака.

In [94]:
theta1, theta2, l1, l2 = sp.symbols('theta1, theta2, l1, l2')
robot = rt.SerialLinkRobot()
robot.add_revolute_joint(theta1, 0, l1, 0)
robot.add_revolute_joint(theta2, 0, l2, 0)
robot.add_subs([(l1, 1), (l2, 1)])
robot.interact()

Откако го конструиравме роботот, ќе ја извадиме конечната DH матрица.

In [95]:
T = robot.get_dh_matrix()
T

Matrix([
[cos(theta1 + theta2), -sin(theta1 + theta2), 0, l1*cos(theta1) + l2*cos(theta1 + theta2)],
[sin(theta1 + theta2),  cos(theta1 + theta2), 0, l1*sin(theta1) + l2*sin(theta1 + theta2)],
[                   0,                     0, 1,                                        0],
[                   0,                     0, 0,                                        1]])

#### Постапка за решавање на проблемот за инверзна кинематика на овој робот, чекор по чекор, користејќи ја sympy.nonlinsolve

Прво ги прегледуваме равенките за инверзна кинематика на овој робот.

In [96]:
xe, ye = T[:2, 3]

In [97]:
xe

l1*cos(theta1) + l2*cos(theta1 + theta2)

In [98]:
ye

l1*sin(theta1) + l2*sin(theta1 + theta2)

Потоа ги испишуваме равенките за sympy да го реши овој систем нелинеарни равенки. Бидејќи решението не се добива директно преку аглите, мора да се користат синусите и косинусите од истите агли за да се добие решение со помош на sympy. Покрај двете равенки за директна кинематика, внесуваме уште две општи тригонометриски равенки за sympy да знае како се поврзани синусот и косинусот на даден агол.

In [99]:
x, y, z = sp.symbols('x, y, z')
s1, c1, s12, c12 = sp.symbols('s1, c1, s12, c12')
equations = [
    l1*c1 + l2*c12 - x,
    l1*s1 + l2*s12 - y,
    s1**2 + c1**2 - 1,
    s12**2 + c12**2 - 1,
]
solutions = sp.nonlinsolve(equations, [s1, c1, s12, c12])
solutions

{((l1**2*y - l2**2*y + x**2*y - x*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4) + y**3)/(2*l1*(x**2 + y**2)), (l1**2*x - l2**2*x + x**3 + x*y**2 + y*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4))/(2*l1*(x**2 + y**2)), (-l1**2*y + l2**2*y + x**2*y + x*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4) + y**3)/(2*l2*(x**2 + y**2)), x*(-l1**2 + l2**2 + x**2 + y**2)/(2*l2*(x**2 + y**2)) - y*sqrt(-(-l1**2 - 2*l1*l2 - l2**2 + x**2 + y**2)*(-l1**2 + 2*l1*l2 - l2**2 + x**2 + y**2))/(2*l2*(x**2 + y**2))), ((l1**2*y - l2**2*y + x**2*y + x*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4) + y**3)/(2*l1*(x**2 + y**2)), (l1**2*x - l2**2*x + x**3 + x*y**2 - y*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x

Сега можеме да тестираме дали е точно решението, но прво мора да ги замениме вредностите за бараната позиција и конструкциските параметри.

In [100]:
solutions_subbed = solutions.subs([(x, 0.7), (y, 1.2), (l1, 1), (l2, 1)])
solutions_subbed

{(0.6 - 0.256306467303007*sqrt(2), 0.35 + 0.439382515376583*sqrt(2), 0.256306467303007*sqrt(2) + 0.6, -0.271380712315168), (0.256306467303007*sqrt(2) + 0.6, 0.35 - 0.439382515376583*sqrt(2), 0.6 - 0.256306467303007*sqrt(2), 0.971380712315168)}

Бидејќи аглите ги претставивме преку синуси и косинуси, сега ќе ги добиете самите агли преку функцијата `atan2()`.

In [101]:
for i, solution in enumerate(solutions_subbed):
    s1n, c1n, s12n, c12n = solution
    theta1n = sp.deg(sp.atan2(s1n, c1n)).evalf(3)
    theta12n = sp.deg(sp.atan2(s12n, c12n)).evalf(3)
    theta2n = theta12n - theta1n
    print(f'Solution {i+1}: {(theta1n, theta2n)}')

Solution 1: (13.7, 92.0)
Solution 2: (106., -92.0)


Решението можеме да го прибереме во една функција, за да изгледа компактно.

In [102]:
def inverse_kinematics(xn, yn):
    x, y, z = sp.symbols('x, y, z')
    s1, c1, s12, c12 = sp.symbols('s1, c1, s12, c12')
    equations = [
        l1*c1 + l2*c12 - x,
        l1*s1 + l2*s12 - y,
        s1**2 + c1**2 - 1,
        s12**2 + c12**2 - 1]
    solutions = sp.nonlinsolve(equations, [s1, c1, s12, c12])
    display('Решенија изразени преку симболички променливи:')
    display(solutions)
    solutions_subbed = solutions.subs([(x, xn), (y, yn), (l1, 1), (l2, 1)])
    for i, solution in enumerate(solutions_subbed):
        s1n, c1n, s12n, c12n = solution
        theta1n = sp.deg(sp.atan2(s1n, c1n)).evalf(3)
        theta12n = sp.deg(sp.atan2(s12n, c12n)).evalf(3)
        theta2n = theta12n - theta1n
        print(f'Решенија изразени преку нумерички вредности {i+1}: {(theta1n, theta2n)}')


inverse_kinematics(xn=0.5, yn=1.7)

'Решенија изразени преку симболички променливи:'

{((l1**2*y - l2**2*y + x**2*y - x*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4) + y**3)/(2*l1*(x**2 + y**2)), (l1**2*x - l2**2*x + x**3 + x*y**2 + y*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4))/(2*l1*(x**2 + y**2)), (-l1**2*y + l2**2*y + x**2*y + x*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4) + y**3)/(2*l2*(x**2 + y**2)), x*(-l1**2 + l2**2 + x**2 + y**2)/(2*l2*(x**2 + y**2)) - y*sqrt(-(-l1**2 - 2*l1*l2 - l2**2 + x**2 + y**2)*(-l1**2 + 2*l1*l2 - l2**2 + x**2 + y**2))/(2*l2*(x**2 + y**2))), ((l1**2*y - l2**2*y + x**2*y + x*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x**2 + 2*l1**2*y**2 - l2**4 + 2*l2**2*x**2 + 2*l2**2*y**2 - x**4 - 2*x**2*y**2 - y**4) + y**3)/(2*l1*(x**2 + y**2)), (l1**2*x - l2**2*x + x**3 + x*y**2 - y*sqrt(-l1**4 + 2*l1**2*l2**2 + 2*l1**2*x

Решенија изразени преку нумерички вредности 1: (46.0, 55.3)
Решенија изразени преку нумерички вредности 2: (101., -55.3)


### Задача 2.61

In [103]:
IFrame(src='https://drive.google.com/file/d/14MN5DozPmAN7WlsDVU8_3OzxF1auEDS0/preview', width='400', height='300')

Во збирката не е наведено дека кое е барањето при инверзната кинематика. Според решението во збирката, се бара одговор на инверзната кинематика за позиција, па на тој начин ќе ја решаваме задача.

Овој тип на задачи се решава на рака. Користете го знаењето стекнато по математика, како знаете и умеете, за да ги изразите вредносите на зглобовите. Тригонометриските идентите многу ќе ви помогнат во решавањето. Интерактивната околина нека ви биде помагало за да проверите дали ви се точни вредностите кои сте ги добиле од пресметките. Дополнително, можете да ја искористите библиотеката `sympy` за упростување на математичките изрази со функцијата `simplify()`. `Sympy` знае да работи со тригонометриски идентитети.

Прво ќе го конструираме роботот со помош на интерактивната околината. DH табелата е веќе дадена.

In [104]:
theta1, theta2, d3, d1 = sp.symbols('theta1, theta2, d3, d1')
robot = rt.SerialLinkRobot()
robot.add_revolute_joint(theta1, d1, 0, -sp.pi/2)
robot.add_revolute_joint(theta2, 0, 0, sp.pi/2)
robot.add_prismatic_joint(0, d3, 0, 0)
robot.add_subs([(d1, 0)])
robot.interact()

Потоа ја добиваме матрицата на директната кинематика. 

In [105]:
T = robot.get_dh_matrix()
T

Matrix([
[cos(theta1)*cos(theta2), -sin(theta1), sin(theta2)*cos(theta1), d3*sin(theta2)*cos(theta1)],
[sin(theta1)*cos(theta2),  cos(theta1), sin(theta1)*sin(theta2), d3*sin(theta1)*sin(theta2)],
[           -sin(theta2),            0,             cos(theta2),        d1 + d3*cos(theta2)],
[                      0,            0,                       0,                          1]])

Од тука ќе ги искористиме изразите за векторот на позиција за да ги определиме вредностите на $\theta_1, \theta_2$ и $d_3$.

In [106]:
x, y, z = T[:3, 3]
y

d3*sin(theta1)*sin(theta2)

Очигледно е дека $\cfrac{y}{x} = \tan(\theta_1)$,

In [107]:
sp.simplify(y / x)

tan(theta1)

па лесно изразуваме $\theta_1 = \arctan2(y, x)$.

Изразот $x^2 + y^2$ се состои од синус, а нема косинус.

In [108]:
sp.simplify(x ** 2 + y ** 2)

d3**2*sin(theta2)**2

Изразот $(z - d_1)^2$ се состои од косинус, а нема синус. 

In [109]:
(z - d1) ** 2

d3**2*cos(theta2)**2

Заклучуваме дека можеме да ги искористиме изразите $x^2 + y^2$ и $(z - d_1)^2$ за да го добиеме тангенсот на $\theta_2$.

In [110]:
sp.simplify((x ** 2 + y ** 2) / (z - d1) ** 2)

tan(theta2)**2

Од тука натаму е лесно, $\theta_2 = \arctan2(\sqrt{x^2 + y^2}, z - d_
1)$

Откако ја знаеме вредноста на $\theta_2$, ќе го изразиме $d_3$ од равенката на $z$.

In [111]:
(z - d1) / sp.cos(theta2)

d3

Се добива $d_3 = \cfrac{z-d_1}{\cos(\theta_2)}$

За крај останува да ги испитате специјалните случаи. На пример, во изразот за $\cfrac{y}{x}$ кратиме со $\sin(\theta_2)$, па треба соодветно да ги пресметаме и толкуваме вредностите на останатите зглобови кога $\sin(\theta_2) = 0$ бидејќи горедобиените равенки не важат во овој специјален случај.

На друг начин, решението може да се добие преку sympy.

In [119]:
def inverse_kinematics(xn, yn, zn):
    x, y, z = sp.symbols('x, y, z')
    s1, c1, s2, c2 = sp.symbols('s1, c1, s2, c2')
    equations = [
        d3*s2*c1 - x,
        d3*s1*s2 - y,
        d1 + d3*c2 - z,
        s1**2 + c1**2 - 1,
        s2**2 + c2**2 - 1]
    solutions = sp.nonlinsolve(equations, [s1, c1, s2, c2, d3])
    display('Решенија изразени преку симболички променливи:')
    display(solutions)
    solutions_subbed = solutions.subs([(x, xn), (y, yn), (z, zn), (d1, 0)])
    for i, solution in enumerate(solutions_subbed):
        s1n, c1n, s2n, c2n, d3n = solution
        theta1n = sp.deg(sp.atan2(s1n, c1n)).evalf(3)
        theta2n = sp.deg(sp.atan2(s2n, c2n)).evalf(3)
        d3n = d3n.evalf(3)
        print(f'Решенија изразени преку нумерички вредности {i+1}: {(theta1n, theta2n, d3n)}')


inverse_kinematics(xn=0, yn=-2.5, zn=1)

'Решенија изразени преку симболички променливи:'

{(-y*sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)/(x**2 + y**2), -x*sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)/(x**2 + y**2), -sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2)), (-d1 + z)/sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2), sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)), (-y*sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)/(x**2 + y**2), -x*sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)/(x**2 + y**2), sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2)), (d1 - z)/sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2), -sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)), (y*sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z + x**2 + y**2 + z**2)/(x**2 + y**2), x*sqrt((x**2 + y**2)/(d1**2 - 2*d1*z + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z + x**2 + y**2 

Решенија изразени преку нумерички вредности 1: (-90.0, -112., -2.69)
Решенија изразени преку нумерички вредности 2: (-90.0, 68.2, 2.69)
Решенија изразени преку нумерички вредности 3: (90.0, -68.2, 2.69)
Решенија изразени преку нумерички вредности 4: (90.0, 112., -2.69)


### Задача 2.62

Во збирката не е наведено дека кое е барањето при инверзната кинематика. Според решението во збирката, се бара одговор на инверзната кинематика за позиција, па на тој начин ќе ја решаваме задача.

Се решава многу слично на 2.61.

Прво ќе го конструираме роботот со помош на интерактивната околината. DH табелата е веќе дадена.

In [122]:
theta1, theta2, d3, d1, d2 = sp.symbols('theta1, theta2, d3, d1, d2')
robot = rt.SerialLinkRobot()
robot.add_revolute_joint(theta1, d1, 0, -sp.pi/2)
robot.add_revolute_joint(theta2, d2, 0, sp.pi/2)
robot.add_prismatic_joint(0, d3, 0, 0)
robot.add_subs([(d1, 1), (d2, 1)])
robot.interact()

Потоа ја добиваме матрицата на директната кинематика. 

In [123]:
T = robot.get_dh_matrix()
T

Matrix([
[cos(theta1)*cos(theta2), -sin(theta1), sin(theta2)*cos(theta1), -d2*sin(theta1) + d3*sin(theta2)*cos(theta1)],
[sin(theta1)*cos(theta2),  cos(theta1), sin(theta1)*sin(theta2),  d2*cos(theta1) + d3*sin(theta1)*sin(theta2)],
[           -sin(theta2),            0,             cos(theta2),                          d1 + d3*cos(theta2)],
[                      0,            0,                       0,                                            1]])

Од тука ќе ги искористиме изразите за векторот на позиција за да ги определиме вредностите на $\theta_1, \theta_2$ и $d_3$.

In [16]:
x, y, z = T[:3, 3]
y

d2*cos(theta1) + d3*sin(theta1)*sin(theta2)

Изразот $x^2 + y^2$ се состои од синус, а нема косинус.

In [17]:
sp.simplify(x**2 + y**2 - d2**2)

d3**2*sin(theta2)**2

Изразот $(z - d_1)^2$ се состои од косинус, а нема синус. 

In [18]:
(z - d1) ** 2

d3**2*cos(theta2)**2

Заклучуваме дека можеме да ги искористиме изразите $x^2 + y^2$ и $(z - d_1)^2$ за да го добиеме тангенсот на $\theta_2$.

In [19]:
sp.simplify((x**2 + y**2 - d2**2) / (z - d1) ** 2)

tan(theta2)**2

Од тука натаму е лесно, $\theta_2 = \arctan2(\sqrt{x^2 + y^2 -d_2^2}, z - d_1)$.

Следува изразувањето на $\theta_1$. Имајќи ги предвид $x$ и $y$, ќе добиеме значително упростени изрази за $x\cos(\theta_1) + y\sin(\theta_1)$  и $x\sin(\theta_1) - y\cos(\theta_1)$.

In [20]:
sp.simplify(x * sp.cos(theta1) + y * sp.sin(theta1))

d3*sin(theta2)

In [21]:
sp.simplify(x * sp.sin(theta1) - y * sp.cos(theta1))

-d2

Понатаму рачно го изразуваме $\sin(\theta_1)$ од едната равенка и го заменуваме во другата за да го изразиме $\cos(\theta_1)$ без да зивиси од $\sin(\theta_1)$. Истото го правиме и за $\sin(\theta_1)$, односно го изразуваме $\sin(\theta_1)$ без да зависи од $\cos(\theta_1)$. На крајот ќе го искористиме нивниот тангенс за да ја изразиме $\theta_1$. Се добива $\theta_1 = \arctan2(y\sin(\theta_2)d_3 - xd_2, x\sin(\theta_2)d_3 + yd_2$.

Откако ја знаеме вредноста на $\theta_2$, ќе го изразиме $d_3$ од равенката на $z$.

In [22]:
(z - d1) / sp.cos(theta2)

d3

Се добива $d_3 = \cfrac{z-d_1}{\cos(\theta_2)}$

За крај останува да ги испитате специјалните случаи.

#### Постапка за решавање на проблемот за инверзна кинематика на овој робот, чекор по чекор, користејќи ја sympy.nonlinsolve

Прво ги прегледуваме равенките за инверзна кинематика на овој робот.

In [124]:
xe, ye, ze = T[:3, 3]

In [125]:
xe

-d2*sin(theta1) + d3*sin(theta2)*cos(theta1)

In [126]:
ye

d2*cos(theta1) + d3*sin(theta1)*sin(theta2)

In [128]:
ze

d1 + d3*cos(theta2)

Потоа ги испишуваме равенките за sympy да го реши овој систем нелинеарни равенки. Бидејќи решението не се добива директно преку аглите, мора да се користат синусите и косинусите од истите агли за да се добие решение со помош на sympy. Покрај двете равенки за директна кинематика, внесуваме уште две општи тригонометриски равенки за sympy да знае како се поврзани синусот и косинусот на даден агол.

In [137]:
x, y, z, s1, c1, s2, c2 = sp.symbols('x, y, z, s1, c1, s2, c2')
subs = [(sp.sin(theta1), s1), (sp.cos(theta1), c1),
        (sp.sin(theta2), s2), (sp.cos(theta2), c2)]
xe, ye, ze = [eq.subs(subs) for eq in [xe, ye, ze]]

In [138]:
equations = [
    xe - x,
    ye - y,
    ze - z,
    s1**2 + c1**2 - 1,
    s2**2 + c2**2 - 1,
]
solutions = sp.nonlinsolve(equations, [s1, c1, s2, c2, d3])
solutions

{((-d2*x - y*sqrt((-d2**2 + x**2 + y**2)/(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))/(x**2 + y**2), (d2*y - x*sqrt((-d2**2 + x**2 + y**2)/(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))/(x**2 + y**2), -sqrt((-d2**2 + x**2 + y**2)/(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2)), (-d1 + z)/sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2), sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2)), ((-d2*x - y*sqrt((-d2**2 + x**2 + y**2)/(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))/(x**2 + y**2), (d2*y - x*sqrt((-d2**2 + x**2 + y**2)/(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))*sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2))/(x**2 + y**2), sqrt((-d2**2 + x**2 + y**2)/(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2)), (d1 - z)/sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2), -sqrt(d1**2 - 2*d1*z - d2**2 + x**2 + y**2 + z**2)), ((-d2*x + y*

Сега можеме да тестираме дали е точно решението, но прво мора да ги замениме вредностите за бараната позиција и конструкциските параметри.

In [142]:
xn, yn, zn = -1, -2.5, 2.5
solutions_subs = solutions.subs([(d1, 1), (d2, 1), (x, xn), (y, yn), (z, zn)])
solutions_subs

{(-0.724137931034483, -0.689655172413793, -0.857492925712544, -0.514495755427527, -2.91547594742265), (-0.724137931034483, -0.689655172413793, 0.857492925712544, 0.514495755427527, 2.91547594742265), (1.0, 0, -0.857492925712544, 0.514495755427527, 2.91547594742265), (1.0, 0, 0.857492925712544, -0.514495755427527, -2.91547594742265)}

Бидејќи аглите ги претставивме преку синуси и косинуси, сега ќе ги добиете самите агли преку функцијата `atan2()`.

In [164]:
q_values = []
for i, solution in enumerate(solutions_subs):
    s1n, c1n, s2n, c2n, d3n = solution
    theta1n = sp.atan2(s1n, c1n)
    theta2n = sp.atan2(s2n, c2n)
    q_value = (
        float(theta1n.evalf(30)),
        float(theta2n.evalf(30)),
        float(d3n.evalf(30)),
    )
    q_values.append(q_value)
    print(f'Решение {i+1} {q_value}')

Решение 1 (-2.3318090810196264, -2.1112158270654806, -2.91547594742265)
Решение 2 (-2.3318090810196264, 1.0303768265243125, 2.91547594742265)
Решение 3 (1.5707963267948966, -1.0303768265243125, 2.91547594742265)
Решение 4 (1.5707963267948966, 2.1112158270654806, -2.91547594742265)


Преку матрицата за директна кинематика можеме да провериме дали решенијата се точни.

In [166]:
for q_value in q_values:
    display(T.subs([*zip([theta1, theta2, d3], q_value)]).subs([(d1, 1), (d2, 1)]))

Matrix([
[0.354824658915535,  0.724137931034483,  0.591374431525893, -1.0],
[0.372565891861312, -0.689655172413793,  0.620943153102187, -2.5],
[0.857492925712544,                  0, -0.514495755427526,  2.5],
[                0,                  0,                  0,    1]])

Matrix([
[-0.354824658915535,  0.724137931034483, -0.591374431525893, -1.0],
[-0.372565891861312, -0.689655172413793, -0.620943153102187, -2.5],
[-0.857492925712544,                  0,  0.514495755427526,  2.5],
[                 0,                  0,                  0,    1]])

Matrix([
[3.1503779002961e-17,                 -1.0, -5.25062983382683e-17, -1.0],
[  0.514495755427526, 6.12323399573677e-17,    -0.857492925712544, -2.5],
[  0.857492925712544,                    0,     0.514495755427526,  2.5],
[                  0,                    0,                     0,    1]])

Matrix([
[-3.1503779002961e-17,                 -1.0, 5.25062983382683e-17, -1.0],
[  -0.514495755427526, 6.12323399573677e-17,    0.857492925712544, -2.5],
[  -0.857492925712544,                    0,   -0.514495755427526,  2.5],
[                   0,                    0,                    0,    1]])

### Задача 2.63

In [2]:
IFrame(src='https://drive.google.com/file/d/14Ar9sDQQjnchG2if5GVwD6OH91BLluho/preview', width='400', height='300')

In [20]:
d1, theta2, theta3, a2, a3 = sp.symbols('d1, theta2, theta3, a2, a3')
robot = rt.SerialLinkRobot()
robot.add_prismatic_joint(0, d1, 0, sp.pi/2)
robot.add_revolute_joint(theta2, 0, a2, -sp.pi/2)
robot.add_revolute_joint(theta3, 0, a3, 0)
robot.add_subs([(a2, 1), (a3, 1)])
robot.interact()

In [5]:
T = robot.get_dh_matrix()
T

Matrix([
[cos(theta2)*cos(theta3), -sin(theta3)*cos(theta2), -sin(theta2),                (a2 + a3*cos(theta3))*cos(theta2)],
[            sin(theta3),              cos(theta3),            0,                                   a3*sin(theta3)],
[sin(theta2)*cos(theta3), -sin(theta2)*sin(theta3),  cos(theta2), a2*sin(theta2) + a3*sin(theta2)*cos(theta3) + d1],
[                      0,                        0,            0,                                                1]])

За разлика од збирката, оваа задача ќе ја решиме на малку поразличен начин. За да ја определиме инверзната кинематика, наместо да ја бараме еднозначно вредноста на секој ротирачки зглоб на раката, овде ќе користиме половични постапки за да ја намалиме комплексноста на аналитичките постапки. На пример, наместо да ги изразуваме $\sin(\theta_x)$ и $\cos(\theta_x)$ сѐ со цел еднозначно да ја определиме $\theta_x$ користејќи ја функцијата `atan2()`, ние ќе ја пресметаме вредноста на $\theta_x$ користејќи само еден од изразите $\sin(\theta_x)$ и $\cos(\theta_x)$, соодветно преку функциите `acos()` и `asin()`. Овие функци не можат еднозначо да ја определат $\theta_x$, па така преку `acos()` ќе имаме две можни решенија зa $\theta_x$, тоа се $\theta_x$ и $-\theta_x$, а преку `asin()` ќе имаме две можни решенија зa $\theta_x$, тоа се $\theta_x$ и $180 - \theta_x$. Сите овие агли не се решенија на проблемот за инверзна кинематика на дадената рака, па ќе треба да се валидира секоја комбинација од вакви агли.

Оваа рака има 3 зглобови, $d_1$, $\theta_2$, $\theta_3$. Во продолжение, за оваа рака, прво ќе ги најдеме двете можни вредности за аголот $\theta_3$, потоа четири можни вредности за $\theta_2$, па на крајот ќе ги најдеме сите можни вредности за $d_1$.

Нека `(xn, yn, zn)` е позицијата која треба да ја постигне роботот. За тест, овде земаме произволни вредносит за xn, yn и zn. Ќе земеме и произволни за константите.

In [12]:
xn, yn, zn = 0.4, 0.6, 0.5
a2n, a3n = 1, 1

In [13]:
x, y, z = T[:3, 3]

In [14]:
y

a3*sin(theta3)

Од равенката за $y$ ќе ја пресметаме $\theta_3$ користејќи `asin()`, па ќе добиеме едно решение од sympy, ама знаме дека и $\sin(180-\theta_3) = \sin(\theta_3)$.

In [15]:
def solve_theta3():
    theta3n = sp.asin(yn/a3n)
    return theta3n, sp.pi - theta3n

theta3n_all = solve_theta3()
theta3n_all

(0.643501108793284, -0.643501108793284 + pi)

Од равенката за $x$ ќе добиеме по две вредности за $\theta_2$ за секоја на $\theta_3$.

In [16]:
x

(a2 + a3*cos(theta3))*cos(theta2)

In [17]:
def solve_theta2(theta3n):
    theta2n = sp.acos(xn / (a2n + a3n*sp.cos(theta3n)))
    return theta2n, -theta2n

theta2n_all = []
for theta3n in solve_theta3():
    theta2n_all.extend(solve_theta2(theta3n))

theta2n_all

[1.34670323449353, -1.34670323449353, 1.31695789692482*I, -1.31695789692482*I]

Во решенијата за $\theta_2$ има и имагинарни вредности. Нив можеме да ги отфрлиме веднаш бидејќи тоа се позиции кои раката не може да ги постигне.

Од равенката за $z$ ќе добиеме по една вредности за секоја комбинација на $\theta_2$ и $\theta_3$.

In [18]:
def solve_d1(theta3n, theta2n):
    d1n = zn - a2n*sp.sin(theta2n) - a3n*sp.sin(theta2n)*sp.cos(theta3n)
    return [d1n]

d1n_all = []
for theta3n in solve_theta3():
    for theta2n in solve_theta2(theta3n):
        d1n_all.extend(solve_d1(theta3n, theta2n))

d1n_all

[-1.25499287747842,
 2.25499287747842,
 0.5 - 0.346410161513776*I,
 0.5 + 0.346410161513776*I]

На крај, ќе итерираме низ сите можни комбинации и ќе ги валидираме сите решенија.

In [19]:
for theta3n in solve_theta3():
    if sp.im(theta3n) != 0:
        print(f'theta3n={theta3n} е комплексен број, па нема да го користиме')
        print('---------------------------------')
        continue
    for theta2n in solve_theta2(theta3n):
        if sp.im(theta2n) != 0:
            print(f'theta2n={theta2n} е комплексен број, па нема да го користиме')
            print('---------------------------------')
            continue
        for d1n in solve_d1(theta3n, theta2n):
            if sp.im(d1n) != 0:
                print(f'd1n={d1n} е комплексен број, па нема да го користиме')
                print('---------------------------------')
                continue
            x_calc = x.subs(d1, d1n).subs(theta2, theta2n).subs(theta3, theta3n).subs(a2, a2n).subs(a3, a3n)
            y_calc = y.subs(d1, d1n).subs(theta2, theta2n).subs(theta3, theta3n).subs(a2, a2n).subs(a3, a3n)
            z_calc = z.subs(d1, d1n).subs(theta2, theta2n).subs(theta3, theta3n).subs(a2, a2n).subs(a3, a3n)
            print(f'Со аглите d1={d1n:.2f}, theta2={theta2n:.2f}, theta3={theta3n:.2f}')
            print(f'стигаме до точката x={x_calc:.2f}, y={y_calc:.2f}, z={z_calc:.2f}')
            print('---------------------------------')

Со аглите d1=-1.25, theta2=1.35, theta3=0.64
стигаме до точката x=0.40, y=0.60, z=0.50
---------------------------------
Со аглите d1=2.25, theta2=-1.35, theta3=0.64
стигаме до точката x=0.40, y=0.60, z=0.50
---------------------------------
theta2n=1.31695789692482*I е комплексен број, па нема да го користиме
---------------------------------
theta2n=-1.31695789692482*I е комплексен број, па нема да го користиме
---------------------------------


Резултатите велат дека имаме две можни решенија од кои првото има вредност има негативна вредност за $d_1$. Ова решение ќе го отфрлиме бидејќи симулацијата моментално не дозолува негативни вредности за призматични зглобови, па испаѓа дека имам само едно решение, $d_1=2.25$, $\theta_2=-1.35$, $\theta_3=0.64$. Проверете го решението во симулацијата. Внимавајте, аглите се изразени во радијани. 

### Задача 2.64

In [120]:
IFrame(src='https://drive.google.com/file/d/1OAzfLFjUtjIWZM1QHkTh2IhSe4IYv6vS/preview', width='400', height='300')

### Задача 2.65

Се решава слично како задача 2.64.